# KNN method

Не смог разобраться с matplotlib ее саму установил но в проге выдает ошибку что не видит модуля с именем matplotlib.Читал как это законтрить но не получилось.Также seaborn.

In [1]:
import pandas as pd
import numpy as np
data_path = '../semeion.data'

Считываем датасет.

In [2]:
def read_data():
    x_digits = []
    y_labels = []
    with open(data_path) as file:  
        for line in file:
            x_digits.append([float(element) for element in line.split()[0:256]])
            y_labels.append(line.split()[256:266].index('1'))
    return np.array(x_digits) , np.array(y_labels)

In [3]:
x , y = read_data()

In [4]:
print(x.shape , y.shape)

(1593, 256) (1593,)


In [5]:
set_labels = pd.DataFrame({'labels': y})
set_labels['labels'].value_counts()

1    162
6    161
4    161
0    161
5    159
3    159
2    159
9    158
7    158
8    155
Name: labels, dtype: int64

Как мы видим количество всех цифр примерно одинаковое

#Нарисуем цифры

In [6]:
#digits = range(10)
#numbers = []
#for i in digits:
    #z = list(map(int, list(y))).index(i)
   #img = x_digits[z].reshape(16,16)
    #fig.add_subplot(1, 10, i + 1)
    #plt.imshow(img)
    #numbers.append(z)
#plt.show

In [7]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 17)

In [9]:
model_knn = KNeighborsClassifier()
model_knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

Посмотрим на результаты.

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,model_knn.predict(x_test))

0.8891213389121339

Настраиваем гиперпараметры нашей модели

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
knn_param = {'n_neighbors':[1,2,3,4,5] + list(range(10,15,1))}

In [13]:
knn_grid = GridSearchCV(model_knn, knn_param,cv = 5)

In [14]:
%%time
knn_grid.fit(x_train, y_train);

CPU times: user 7.08 s, sys: 12.4 ms, total: 7.1 s
Wall time: 7.09 s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 10, 11, 12, 13, 14]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [15]:
knn_grid.best_score_ , knn_grid.best_params_

(0.9013452914798207, {'n_neighbors': 1})

Посмотрим как работает наша модель с найденным параметром на тестововой выборке

In [16]:
knn_train_pred = knn_grid.predict(x_test)

In [17]:
accuracy_score(y_test, knn_train_pred)

0.899581589958159

Наша точность увеличилась примерно на 0.01